In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as const
import matplotlib as mpl
from jupyterthemes import jtplot #These two lines can be skipped if you are not using jupyter themes
jtplot.reset()

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=67.4, Om0=0.314)
import scipy as sp
import multiprocessing as mp

from matplotlib.ticker import StrMethodFormatter
import time
start_total = time.time()

In [2]:
import os
my_path = '/home/tomi/Documentos/Fisica/Tesis/escrito-tesis/images/'

In [3]:
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.lens_model_extensions import LensModelExtensions
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver

In [4]:
zl = 0.2; zs = 1.2
Dl = cosmo.angular_diameter_distance(zl)   
Ds = cosmo.angular_diameter_distance(zs)    
Dls = cosmo.angular_diameter_distance_z1z2(zl, zs)
G = const.G
rho_crit = (cosmo.critical_density(zl)).to(u.kg/u.m**3)
c_light = (const.c).to(u.cm/u.second)

#r0 = 10*u.kpc
r0 = 10.0*u.kpc
#r0 = 0.1*u.kpc
pi = np.pi

def scale_radius(v,Dl,Ds,Dls):                               #this is e0 in eq 3.42 meneghetti, eq 1 barnacka 2014
    return (4.*pi*v**2/c_light**2*Dl*Dls/Ds).decompose()
def theta_E_SIS():
    'in arcsec'
    pre_theta_E = (scale_radius(v,Dl,Ds,Dls)/Dl).decompose()
    return pre_theta_E*u.rad.to('arcsec', equivalencies=u.dimensionless_angles()) 

v = 180 *u.km/u.s
ss_r = scale_radius(v,Dl,Ds,Dls) 
print('scale radius (m): ',ss_r)
print('scale radius (kpc): ',ss_r.to(u.kpc))
print('theta_E: ',theta_E_SIS() ,'arcsec')
theta_E_num = theta_E_SIS()
re = (const.e.esu**2/const.m_e/(c_light**2)).decompose()
print('Classic electron radius: ',re)

scale radius (m):  7.705329461274929e+19 m
scale radius (kpc):  2.49712721364453 kpc
theta_E:  0.7301786792241515 arcsec
Classic electron radius:  2.817940324670788e-15 m


In [5]:
nu = 130.0 * u.MHz
omega = nu*(1.+zl)*2*np.pi
n0 = 300./(u.cm)**(3)

In [6]:
theta_0 = (r0/Dl).decompose()*u.rad.to('arcsec', equivalencies=u.dimensionless_angles()) 
print('Angular scale radius (theta_0 = r0/Dl): ',theta_0, 'arcsec')

Angular scale radius (theta_0 = r0/Dl):  2.9240748137875747 arcsec


In [7]:
from lenstronomy.LensModel.Profiles.plasma_exponential_sie import PlasmaExponentialSIE
perfil = PlasmaExponentialSIE()

In [19]:
def image_position_3(x, y, eta):
    x_s, y_s = x, y
    x_pre, y_pre = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, \
                    sourcePos_x=x_s, sourcePos_y=y_s, min_distance=0.05, search_window=5, \
                    precision_limit=10**(-8), num_iter_max=100,non_linear=True)
    print(x_pre)
    print(y_pre)
    mag = [0,0]
    t = [0,0]
    for k in range(len(x_pre)):
        mag[k] = lensModel.magnification(x_pre[k], y_pre[k], kwargs = kwargs_lens_list)
        phi = PlasmaExponentialSIE.function(perfil, x_pre[k], y_pre[k], theta_E_num.value, eta, A, B, C ,psi0_plasma.value, theta_0.value)
        t[k] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (x_s - x_pre[k] )**2 + (y_s - y_pre[k])**2) - phi**2)*(u.arcsec**2).to('rad**2')).to('s').value
    r = np.abs(mag[0]/mag[1])
    rr = [r,r]
    dt = np.abs(t[0] - t[1])/24/60/60 #convert seconds to days
    dt = [dt,dt]
    return x_pre, y_pre, rr, dt

def main_3(x1,x2):
    start_p = time.time()
    x = x1
    y = x2
    r = []
    dt = []
    
    with mp.Pool(mp.cpu_count()-1) as pool:
        out = pool.map(image_position_2, zip(x,y))
        
        xx = []; yy = []
        for i in range(len(out)):
            x = out[i][0]
            xx.append(x)
            y = out[i][1]
            yy.append(y)
            mag = out[i][2]
            r.append(mag)
            t = out[i][3]
            dt.append(t)
        xx = np.concatenate(xx, axis=0)
        yy = np.concatenate(yy, axis=0)
        r = np.concatenate(r, axis = 0)
        dt = np.concatenate(dt, axis = 0)
    end_p = time.time()
    pool.close()
    print('time: ',(end_p-start_p)/60.,' minutes')
        
    return xx, yy, r, dt

In [9]:
lens_model_list = ['PlasmaExponentialSIE']
lensModel = LensModel(lens_model_list)
lensEquationSolver = LensEquationSolver(lensModel)
A = 2.00744300029693
B = 1.5887225542022732
C = 1.4308204791467736
psi0_plasma = (A*Dls/Ds/Dl*2*pi*c_light**2/omega**2*re*n0*r0 \
            *(u.rad**2).to('arcsec**2', equivalencies=u.dimensionless_angles()) ).decompose()

In [21]:
eta_arr = np.linspace(0, .5, 200)
r_arr = np.linspace(.3, 3, 200)
x_arr = y_arr = r_arr*np.cos(np.pi/4)

In [24]:
x_line_arr = []
y_line_arr = []
result = []
for x in x_arr:
    for i in eta_arr:
        kwargs = {'theta_E':theta_E_num.value,'eta':i, 'A':A, 'B':B, 'C':C ,'psi0_plasma':psi0_plasma.value,\
                        'theta_0':theta_0.value}
        kwargs_lens_list = [kwargs]
        x_image, y_image, mag, dt = image_position_3(x, x, i)
        tmp = [x, i, mag[0], dt[0]]
        result.append(tmp)
        print(x,i)

[ 0.68164679 -0.27024127]
[ 0.68164679 -0.27024127]
0.21213203435596426 0.0
[ 0.67746975 -0.27188494]
[ 0.68580613 -0.26857992]
0.21213203435596426 0.002512562814070352
[ 0.67327714 -0.27351024]
[ 0.68994568 -0.26690164]
0.21213203435596426 0.005025125628140704
[ 0.66907112 -0.27511644]
[ 0.69406339 -0.26520719]
0.21213203435596426 0.007537688442211055
[ 0.6648539  -0.27670289]
[ 0.69815731 -0.26349732]
0.21213203435596426 0.010050251256281407
[ 0.66062768 -0.27826891]
[ 0.70222551 -0.26177283]
0.21213203435596426 0.01256281407035176
[ 0.65639472 -0.27981388]
[ 0.70626616 -0.26003449]
0.21213203435596426 0.01507537688442211
[ 0.65215722 -0.28133719]
[ 0.7102775  -0.25828312]
0.21213203435596426 0.017587939698492462
[ 0.64791743 -0.28283826]
[ 0.71425786 -0.25651951]
0.21213203435596426 0.020100502512562814
[ 0.64367755 -0.28431654]
[ 0.71820566 -0.25474448]
0.21213203435596426 0.022613065326633167
[ 0.63943977 -0.28577149]
[ 0.7221194  -0.25295885]
0.21213203435596426 0.025125628140703

[ 0.39118533 -0.32230396]
[ 0.91745926 -0.12330326]
0.21213203435596426 0.228643216080402
[ 0.38935826 -0.32198124]
[ 0.9188667  -0.12214322]
0.21213203435596426 0.23115577889447236
[ 0.38755284 -0.32164652]
[ 0.92026184 -0.12099401]
0.21213203435596426 0.23366834170854273
[ 0.38576876 -0.32129999]
[ 0.92164496 -0.11985553]
0.21213203435596426 0.23618090452261306
[ 0.38400567 -0.32094184]
[ 0.92301631 -0.11872768]
0.21213203435596426 0.23869346733668342
[ 0.38226326 -0.32057226]
[ 0.92437617 -0.11761038]
0.21213203435596426 0.24120603015075376
[ 0.38054121 -0.32019141]
[ 0.92572477 -0.11650351]
0.21213203435596426 0.24371859296482412
[ 0.3788392  -0.31979949]
[ 0.92706237 -0.11540699]
0.21213203435596426 0.24623115577889448
[ 0.37715695 -0.31939665]
[ 0.9283892  -0.11432071]
0.21213203435596426 0.24874371859296482
[ 0.37549412 -0.31898307]
[ 0.9297055  -0.11324457]
0.21213203435596426 0.25125628140703515
[ 0.37385045 -0.31855892]
[ 0.9310115  -0.11217848]
0.21213203435596426 0.25376884

IndexError: list assignment index out of range

In [ ]:
end_total = time.time()
print('total time: ',(end_total-start_total)/60.,' minutes')